# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [3]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [4]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [5]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

> Note: The 'llama-3.1-70b-versatil' model was deprecated. As a result, we updated it to the 'llama-3.3-70b-versatil' model in January 2025.

In [6]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.3-70b-versatile"

## Creating Crew, Agents, and Tasks

In [7]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2025-08-24 21:22:58,833 - 140031803808640 - sqlite.py-sqlite:111 - WARNING: ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


## Kicking off the Crew

In [9]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2025'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2025. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2025.



# Agent: Lead Market Analyst
## Thought: To create a detailed summary report highlighting the most impactful financial news and updates related to Inflation in the US and the impact on the stock market in 2025, I need to gather the latest information on this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation news 2025 and stock market impact\"}"
## Tool 



# Agent: Lead Market Analyst
## Thought: in app
Open links In app
Thought: Based on the article from CBS News, the US stock market is rallying toward records after data suggested inflation was a touch better than expected last month.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.wsj.com/livecoverage/cpi-report-today-inflation-stock-market-08-12-2025\"}"
## Tool Output: 
wsj.comPlease enable JS and disable any ad blocker


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Search the internet(**kwargs: Any) -> Any
Tool Description: Search the internet(search_query: 'string') - A tool that can be used to search the internet with a search_query. search_query: 'Mandatory search query you want to use to search the internet'
Tool Arguments: {'name': FieldInfo(annotation=str, required=True), 'description': FieldInfo(annotation=str, required=False, default=''), 'args_schema': FieldInfo(annot



# Agent: Lead Market Analyst
## Final Answer: 
The current inflation rate in the US is 2.7% as of July 2025, which is below the expected 2.8%. This has led to a rally in the stock market, with the S&P 500 rising 1.1% and the Dow Jones Industrial Average up 473 points. The Federal Reserve is expected to cut interest rates for the first time this year in September, which could further boost the stock market.

However, there are concerns about the impact of inflation on the stock market, with some investors warning that high inflation could lead to a market correction. The cost of imports is expected to rise due to tariffs, which could lead to higher prices for consumers and impact economic growth.

The mid-year market outlook for 2025 predicts that EM growth will slow to a 2.4% annualized rate in the second half of the year, and EM central banks will likely continue cutting rates despite the Fed. The US economy faces challenges in the latter half of 2025, including tariff-induced infla



# Agent: Chief Data Strategist
## Final Answer: 
The current inflation rate in the US is 2.7% as of July 2025, which is below the expected 2.8%. This has led to a rally in the stock market, with the S&P 500 rising 1.1% and the Dow Jones Industrial Average up 473 points. The Federal Reserve is expected to cut interest rates for the first time this year in September, which could further boost the stock market.

However, there are concerns about the impact of inflation on the stock market, with some investors warning that high inflation could lead to a market correction. The cost of imports is expected to rise due to tariffs, which could lead to higher prices for consumers and impact economic growth.

The mid-year market outlook for 2025 predicts that EM growth will slow to a 2.4% annualized rate in the second half of the year, and EM central banks will likely continue cutting rates despite the Fed. The US economy faces challenges in the latter half of 2025, including tariff-induced inf



# Agent: Creative Content Director
## Final Answer: 
The final content includes a complete blog post with key insights and strategies, alongside social media updates to educate the audience about the inflation landscape and stock market impacts in 2025.


# Agent: Chief Content Officer
## Task: Review and refine the content created on Inflation in the US and the impact on the stock market in 2025 to ensure it meets the highest standards of accuracy, clarity, and brand alignment. Thoroughly proofread and edit the content, checking for errors, inconsistencies, and alignment with the brand voice. Ensure that the content accurately reflects the key insights and recommendations provided by the Data Analyst and Market News Monitor agents. Ensure that the final content is well-formatted in markdown, using appropriate headers, bullet points, links, and other markdown features to enhance readability and engagement.



# Agent: Chief Content Officer
## Final Answer: 
# Understanding Inflation 

## Social Content

In [10]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
As inflation trends continue to evolve,
understanding their impact on the stock market is
essential for making informed decisions in 2025! 📈
#InvestSmart #InflationImpact
--------------------------------------------------
LinkedIn
Looking ahead to 2025, inflation poses both
challenges and opportunities for investors.
Discover key strategies to prepare your portfolio
effectively for the inflation landscape. [Link to
full article]
--------------------------------------------------


## Blog Post

In [11]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Understanding Inflation in the US and Its Impact on the Stock Market in 2025

As we look ahead to 2025, understanding inflation and its implications for the stock market becomes increasingly crucial for investors. Inflation, a general increase in prices and fall in the purchasing value of money, can significantly influence market dynamics. By examining current trends and expert insights, we can provide actionable recommendations for navigating this evolving financial landscape.

## Current State of Inflation

In the wake of recent economic shifts, the U.S. is experiencing fluctuating inflation rates. As of October 2023, inflation remains a prominent concern, primarily driven by factors such as supply chain disruptions, labor market constraints, and changes in consumer demand. According to the latest reports from the Bureau of Labor Statistics, inflation rates have consistently shown a year-over-year increase, prompting the Federal Reserve to adapt its monetary policy accordingly.

Experts suggest that these inflationary pressures will continue into 2025, albeit at potentially varying levels. Economists project that inflation may stabilize as supply chains rebalance and consumer behavior adjusts. However, understanding the nuances of how inflation interacts with various sectors of the economy is essential for making informed investment decisions.

### Key Insights on the Stock Market Impact

The relationship between inflation and the stock market is complex. Historically, moderate inflation can be a sign of a growing economy, potentially leading to increased corporate earnings. However, excessive inflation can erode purchasing power and lead to rising costs for businesses, ultimately impacting their bottom lines. In 2025, investors should closely monitor sectors that are either inflation-resistant or likely to benefit from price increases.

1. **Sector Performance**  
   - **Consumer Staples:** Companies in this sector, which include essential goods producers, often maintain stability during inflation due to consistent demand.  
   - **Real Estate:** Inflation can boost real estate values, making REITs an attractive option for those seeking to hedge against inflation.

2. **Investment Strategies**  
   - **Diversification:** A well-diversified portfolio can help mitigate risks associated with inflation. Consider incorporating a mix of equities, fixed income, and commodities.  
   - **Focus on Dividend Stocks:** Firms that consistently pay and grow dividends may provide a buffer against inflation, as these payouts often increase over time.

## Actionable Recommendations for Investors

To effectively navigate the expected inflation landscape and the stock market's response in 2025, investors should consider the following strategies:

- **Stay Informed:** Regularly review economic indicators, including the Consumer Price Index (CPI) and Producer Price Index (PPI), which provide insight into inflationary trends.  
- **Adjust Asset Allocation:** As inflation rates change, it may be wise to adjust your asset allocation. This could mean increasing exposure to inflation-sensitive assets while reducing allocation to areas that may be adversely affected.

Investors should also engage with reliable financial news sources and market analysts to stay ahead of developments that may impact their strategies. Utilizing platforms that provide real-time insights can foster more informed decision-making.

### Conclusion

In conclusion, inflation is set to remain a key factor influencing the stock market in 2025. With careful analysis and strategic planning, investors can position themselves to not only withstand the effects of inflation but also take advantage of potential opportunities that arise. By remaining informed and adaptable, you can navigate these complexities with confidence.

---

### Social Media Updates

- **Twitter Update:** "As inflation trends continue to evolve, understanding their impact on the stock market is essential for making informed decisions in 2025! 📈 #InvestSmart #InflationImpact"  
  
- **LinkedIn Article Highlight:** "Looking ahead to 2025, inflation poses both challenges and opportunities for investors. Discover key strategies to prepare your portfolio effectively for the inflation landscape. [Link to full article]"